In [1]:
import h5py
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import *
from sklearn.model_selection import KFold

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6658158891217609491
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5060693856
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7398423959215979180
physical_device_desc: "device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [3]:
TRAIN_PATH = r'.\data\train.mp4'
LABEL_PATH = r'.\data\train.txt'
H5PY_PATH  = r'.\data\train.h5'

In [4]:
class DataGenerator(keras.utils.Sequence):

    def __init__(
        self, data_path, indexes=None, n_channels=6, batch_size=16, shuffle=True
    ):
        self.batch_size = batch_size
        self.data_path = data_path
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.file = h5py.File(data_path, 'r')
        if indexes is None:
            self.indexes = np.arange(len(self.file['FRM']))
        else:
            self.indexes = indexes

    def __len__(self):
        return int(np.floor(len(self.indexes) / self.batch_size))

    def __getitem__(self, index):
        X, y = self.__data_generation(
            list(self.indexes[index*self.batch_size:(index+1)*self.batch_size])
        )
        return X, y

    def __del__(self):
        self.file.close()

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, index):
        index = list(index)
        index.sort()
        X = np.array(self.file['FRM'][index])
        y = np.array(self.file['LBL'][index])
        return X, y

In [5]:
class SpeedNet(keras.Model):

    def __init__(self, input_dim=(150, 300, 2), kernel_size=3):
        super(SpeedNet, self).__init__()
        self.modules = Sequential([
            Conv2D(8, (kernel_size, kernel_size), activation='relu', data_format='channels_last'),
            MaxPooling2D(pool_size=(1, 2)),
            Conv2D(16, (kernel_size, kernel_size), activation='relu', data_format='channels_last'),
            MaxPooling2D(pool_size=(1, 2)),
            Conv2D(32, (kernel_size, kernel_size), activation='relu', data_format='channels_last'),
            MaxPooling2D(),
            Conv2D(64, (kernel_size, kernel_size), activation='relu', data_format='channels_last'),
            MaxPooling2D(),
            Conv2D(128, (kernel_size, kernel_size), activation='relu', data_format='channels_last'),
            MaxPooling2D(),
            Conv2D(256, (kernel_size, kernel_size), activation='relu', data_format='channels_last'),
            MaxPooling2D(),
            Conv2D(512, (kernel_size, kernel_size), activation='relu', data_format='channels_last', padding='same'),
            MaxPooling2D(),
            GlobalAveragePooling2D(),
            Dropout(0.2),
            Dense(500, activation='relu'),
            Dense(250, activation='relu'),
            Dense(1, activation='relu')
        ])

    def call(self, x):
        x = self.modules(x)
        return x

In [6]:
TRAIN_COUNT = 20399

kf = KFold(n_splits=5)

valid_loss = []
fold_var = 0

for train_index, val_index in kf.split(np.arange(TRAIN_COUNT)):
    train_data_generator = DataGenerator(
        H5PY_PATH, indexes=train_index, batch_size=32
    )
    valid_data_generator = DataGenerator(
        H5PY_PATH, indexes=val_index, batch_size=32
    )

    model = SpeedNet()
    model.compile(loss='mse', optimizer='adam')

    es = keras.callbacks.EarlyStopping(
        monitor='val_loss',
        mode='min',
        verbose=1,
        patience=3
    )
    mc = keras.callbacks.ModelCheckpoint(
        f'./models/best_model_{fold_var}.h5',
        monitor='val_loss',
        mode='min',
        save_best_only=True,
        verbose=1
    )

    history = model.fit(
        train_data_generator,
        epochs=100,
        callbacks=[es, mc],
        validation_data=valid_data_generator
    )

    model.load_weights(f'./models/best_model_{fold_var}.h5')

    results = model.evaluate(valid_data_generator)
    valid_loss.append(results)

    keras.backend.clear_session()

    fold_var += 1

InvalidArgumentError: Value for attr 'T' of uint8 is not in the list of allowed values: half, bfloat16, float, double, int32
	; NodeDef: {{node Conv2D}}; Op<name=Conv2D; signature=input:T, filter:T -> output:T; attr=T:type,allowed=[DT_HALF, DT_BFLOAT16, DT_FLOAT, DT_DOUBLE, DT_INT32]; attr=strides:list(int); attr=use_cudnn_on_gpu:bool,default=true; attr=padding:string,allowed=["SAME", "VALID", "EXPLICIT"]; attr=explicit_paddings:list(int),default=[]; attr=data_format:string,default="NHWC",allowed=["NHWC", "NCHW"]; attr=dilations:list(int),default=[1, 1, 1, 1]> [Op:Conv2D]